In [42]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
from pathlib import Path


import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# ==========================================================
# CONFIGURATION
# ==========================================================
np.random.seed(42)
torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Device: {device}")

# Paths
ROOT = Path("Amazon_products")
TRAIN_CORPUS_PATH = ROOT / "train" / "train_corpus.txt"
TEST_CORPUS_PATH  = ROOT / "test" / "test_corpus.txt"
CLASS_PATH        = ROOT / "classes.txt"

EMB_DIR          = Path("Embeddings")
X_ALL_PATH       = EMB_DIR / "X_train_test_mpn.pt"
LABEL_EMB_PATH   = EMB_DIR / "labels_base_new_mpn.pt"

MODEL_SAVE = Path("Models")
MODEL_SAVE.mkdir(exist_ok=True)
MODEL_PATH = MODEL_SAVE / "silver_classifier.pt"

# ==========================================================
# LOAD IDS
# ==========================================================
def load_ids(path):
    ids = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            pid, _ = line.strip().split("\t", 1)
            ids.append(int(pid))
    return ids

train_ids = load_ids(TRAIN_CORPUS_PATH)
test_ids  = load_ids(TEST_CORPUS_PATH)
n_train = len(train_ids)
n_test  = len(test_ids)

print(f"Train IDs: {n_train} | Test IDs: {n_test}")

# ==========================================================
# LOAD SILVER LABELS
# ==========================================================
with open("Silver/silver_train_new_mpn_nohier.json", "r", encoding="utf-8") as f:
    raw = json.load(f)

silver_labels = {int(pid): data["labels"] for pid, data in raw.items()}

# ==========================================================
# LOAD X_all
# ==========================================================
print("\n🧠 Loading X_all.pt ...")
data = torch.load(X_ALL_PATH, weights_only=False)

if isinstance(data, np.ndarray):
    data = torch.from_numpy(data)
elif isinstance(data, list):
    data = torch.stack(data)

X_all = data.float().to(device)
assert X_all.shape[0] == n_train + n_test

X_train = X_all[:n_train]
X_test  = X_all[n_train:]
print(f"✓ X_train: {X_train.shape} | X_test: {X_test.shape}")

# ==========================================================
# LOAD LABEL EMBEDDINGS
# ==========================================================
tmp = torch.load(LABEL_EMB_PATH, weights_only=False)

# Convertir numpy → tensor si nécessaire
if isinstance(tmp, np.ndarray):
    tmp = torch.from_numpy(tmp)

label_emb = tmp.float().to(device)
print(f"✓ Label embeddings: {label_emb.shape}")

# ==========================================================
# LOAD CLASS NAMES
# ==========================================================
classes = {}
with open(CLASS_PATH, "r", encoding="utf-8") as f:
    for line in f:
        cid, cname = line.strip().split("\t")
        classes[int(cid)] = cname

n_classes = len(classes)

pid2idx = {pid: i for i, pid in enumerate(train_ids)}


🔧 Device: cuda
Train IDs: 29487 | Test IDs: 19658

🧠 Loading X_all.pt ...
✓ X_train: torch.Size([29487, 768]) | X_test: torch.Size([19658, 768])
✓ Label embeddings: torch.Size([531, 768])


In [43]:
class MultiLabelDataset(Dataset):
    def __init__(self, pids, labels_dict):
        self.pids = pids
        self.labels = labels_dict

    def __len__(self):
        return len(self.pids)

    def __getitem__(self, idx):
        pid = self.pids[idx]
        emb = X_train[pid2idx[pid]]

        y = torch.zeros(n_classes)
        for c in self.labels[pid]:
            if 0 <= c < n_classes:
                y[c] = 1.0

        return {"X": emb, "y": y}


In [44]:
train_p, val_p = train_test_split(
    list(silver_labels.keys()), test_size=0.2, random_state=42
)

train_dataset = MultiLabelDataset(train_p, silver_labels)
val_dataset   = MultiLabelDataset(val_p, silver_labels)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64)

In [45]:
class InnerProductClassifier(nn.Module):
    def __init__(self, input_dim, label_embeddings, dropout=0.2, trainable_label_emb=False):
        super().__init__()

        D = label_embeddings.size(1)

        self.proj = nn.Linear(input_dim, D)
        self.dropout = nn.Dropout(dropout)

        if trainable_label_emb:
            self.label_emb = nn.Parameter(label_embeddings.clone())
        else:
            self.register_buffer("label_emb", label_embeddings.clone())

    def forward(self, x, use_dropout=True):
        if use_dropout:
            x = self.dropout(x)

        x_proj = self.proj(x)                 # (B, D)
        logits = x_proj @ self.label_emb.T    # (B, C)

        return logits

In [46]:
def evaluate(model, loader, thr=0.25):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for batch in loader:
            X = batch["X"]
            y = batch["y"].numpy()

            prob = torch.sigmoid(model(X)).cpu().numpy()
            pred = (prob > thr).astype(int)

            preds.extend(pred)
            labels.extend(y)

    f1s = f1_score(labels, preds, average="samples")
    f1m = f1_score(labels, preds, average="macro")
    return f1s, f1m

In [47]:
import copy
print("\n🚀 Training (Mean Teacher Regularization)...")

# ----------------------------
# Hyperparameters
# ----------------------------
epochs = 100
patience = 5
wait = 0
best_f1 = 0

alpha_ema = 0.995       # teacher EMA speed
lambda_cons = 1.5      # weight for consistency loss
noise_std = 0.05        # noise on student input

# ----------------------------
# Init student + teacher
# ----------------------------
student = InnerProductClassifier(
    input_dim=X_train.size(1),
    label_embeddings=label_emb,
    dropout=0.2,
    trainable_label_emb=False
).to(device)

teacher = copy.deepcopy(student).to(device)

optimizer = torch.optim.AdamW(student.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

best_teacher = copy.deepcopy(teacher.state_dict())

# ----------------------------
# Consistency loss
# ----------------------------
def consistency_loss(log_s, log_t):
    ps = torch.sigmoid(log_s)
    pt = torch.sigmoid(log_t)
    return F.mse_loss(ps, pt)

# ----------------------------
# Training loop
# ----------------------------
for epoch in range(1, epochs + 1):
    student.train()
    teacher.eval()

    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        X = batch["X"].to(device)
        y = batch["y"].to(device)

        # Add noise to student
        noisy_X = X + noise_std * torch.randn_like(X)

        # student forward
        logits_s = student(noisy_X)

        # teacher forward (no gradient)
        with torch.no_grad():
            logits_t = teacher(X)

        # supervised = main objective
        loss_sup = F.binary_cross_entropy_with_logits(logits_s, y)

        # consistency = stability objective
        loss_cons = consistency_loss(logits_s, logits_t)

        # total loss
        loss = loss_sup + lambda_cons * loss_cons

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # EMA update (teacher → student)
        for t_param, s_param in zip(teacher.parameters(), student.parameters()):
            t_param.data = alpha_ema * t_param.data + (1 - alpha_ema) * s_param.data

        total_loss += loss.item()

    scheduler.step()


    teacher.eval()
    f1_sample, f1_macro = evaluate(teacher, val_loader)

    print(f"[Epoch {epoch}] Loss={total_loss/len(train_loader):.4f} | F1={f1_sample:.4f}")

    if f1_sample > best_f1:
        best_f1 = f1_sample
        best_teacher = copy.deepcopy(teacher.state_dict())
        wait = 0
        print(f"New best model saved (F1={best_f1:.4f})")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered")
            break


teacher.load_state_dict(best_teacher)
print("\n🎉 Final best F1:", best_f1)



🚀 Training (Mean Teacher Regularization)...


Epoch 1:   0%|          | 0/369 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 369/369 [00:01<00:00, 189.66it/s]


[Epoch 1] Loss=0.1633 | F1=0.0010
New best model saved (F1=0.0010)


Epoch 2: 100%|██████████| 369/369 [00:01<00:00, 213.04it/s]


...

[Epoch 94] Loss=0.0123 | F1=0.5931


Epoch 95: 100%|██████████| 369/369 [00:01<00:00, 310.64it/s]


[Epoch 95] Loss=0.0123 | F1=0.5932


Epoch 96: 100%|██████████| 369/369 [00:01<00:00, 266.47it/s]


[Epoch 96] Loss=0.0123 | F1=0.5929
Early stopping triggered

🎉 Final best F1: 0.5933874544505268


In [ ]:
import csv
import numpy as np
from pathlib import Path

print("\nGenerating submission...")

teacher.eval()

X_test = X_test.to(device)

def select_k(prob, min_k=2, max_k=3):
    idx = np.argsort(prob)[::-1]  # descend
    top3 = idx[:max_k]

    if prob[top3[2]] < 0.25 * prob[top3[1]]:
        return top3[:2]

    return top3


preds = []

with torch.no_grad():
    for start in tqdm(range(0, len(X_test), 64)):
        batch = X_test[start:start+64]
        logits = teacher(batch, use_dropout=False)

        probs = torch.sigmoid(logits).cpu().numpy()

        for p in probs:
            labels = select_k(p)
            preds.append([str(x) for x in labels])


# ==========================================================
# SAVE CSV
# ==========================================================

OUT_DIR = Path("Submission")
OUT_DIR.mkdir(exist_ok=True)
OUT_PATH = OUT_DIR / "submission2_inner.csv"

with open(OUT_PATH, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["id", "label"])
    for pid, labels in zip(test_ids, preds):
        w.writerow([pid, ",".join(labels)])

print(f"🎉 Submission saved → {OUT_PATH}")



Generating submission...


100%|██████████| 308/308 [00:00<00:00, 965.47it/s]

🎉 Submission saved → Submission\submission2_inner.csv


: 